<a href="https://colab.research.google.com/github/ahoiching/FDM/blob/main/FDM_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%shell
pip install pandas --upgrade
pip install scipy --upgrade
pip install tqdm --upgrade
pip install --upgrade cvxpy
pip install cobra

wget https://packages.gurobi.com/9.5/gurobi9.5.2_linux64.tar.gz -O /opt/gurobi9.5.2_linux64.tar.gz
tar xfvz /opt/gurobi9.5.2_linux64.tar.gz -C /opt/
pip install gurobipy==9.5.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 33.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: cvxpy
    Found existing installation: 

In [4]:
#Download the academic license file from Web License Service: https://www.gurobi.com/features/web-license-service/
#Here I downloaded the GUROBI License file to my Dropbox and created a Dropbox link for wget.

#!mkdir /opt/gurobi
#!wget https://www.dropbox.com/s/GUROBI_LICENSE_LINK -O /opt/gurobi/gurobi.lic

In [3]:
!git clone https://github.com/ahoiching/FDM.git

Cloning into 'FDM'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 72 (delta 34), reused 7 (delta 3), pack-reused 0
Unpacking objects: 100% (72/72), 564.50 KiB | 2.97 MiB/s, done.


In [5]:
%cd ./FDM/
import FDM
import pandas as pd
import numpy as np

/content/FDM


In [6]:
(flux_modes, flux_modes_curate, flux_decomposition, functional_decomposition,
  gene_expressions_decomposition)=FDM.Get_yield_flux_functional_gene_decomposition_from_file('./model_example.json')

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID
Academic license - for non-commercial use only - registered to billywcheng@gmail.com


100%|██████████| 1877/1877 [00:00<00:00, 1031404.25it/s]



.
.
Metabolites loaded...


100%|██████████| 2791/2791 [00:00<00:00, 15613.30it/s]



.
.
Reactions loaded...


...


.
.
Getting gene reactions matrix from FBA model...

.
.
Genes loaded...


100%|██████████| 2791/2791 [00:00<00:00, 215657.17it/s]



.
.
Reactions loaded...
Academic license - for non-commercial use only - registered to billywcheng@gmail.com
Academic license - for non-commercial use only - registered to billywcheng@gmail.com
Reactions that are constraint to carry fixed non-zero fluxes:
Lower bound		Upper bound 		Reaction ID 
5.3024922118  		 5.3024922118 		 EX_ac_e
72.1076287777  		 72.1076287777 		 ATPM
0.130505201  		 0.130505201 		 Sink_RNA|atp_c
0.1214848392  		 0.1214848392 		 Sink_RNA|ctp_c
0.1643765566  		 0.1643765566 		 Sink_RNA|gtp_c
0.1067840295  		 0.1067840295 		 Sink_RNA|utp_c
0.5564993963  		 0.5564993963 		 Sink_Protein|ala__L_c
0.294656916  		 0.294656916 		 Sink_Protein|arg__L_c
0.2253865467  		 0.2253865467 		 Sink_Protein|asn__L_c
0.3195778331  		 0.3195778331 		 Sink_Protein|asp__L_c
0.0440936425  		 0.0440936425 		 Sink_Protein|cys__L_c
0.2132579501  		 0.2132579501 		 Sink_Protein|gln__L_c
0.3612168677  		 0.3612168677 		 Sink_Protein|glu__L_c
0.4320056384  		 0.4320056384 		 Sink_Protein|gly

100%|██████████| 81/81 [00:28<00:00,  2.89it/s]



..
The condition is identified as aerobic E.coli..
..
..



In [7]:
protein_mass_fractions=pd.read_csv('./example_protein_mass_fractions.csv', 
                                   index_col=0)
shared_index=np.intersect1d(protein_mass_fractions.index, 
                            gene_expressions_decomposition.index)
non_FDM_index=np.setdiff1d(protein_mass_fractions.index, 
                          gene_expressions_decomposition.index)

In [8]:
FDM_decomposition=gene_expressions_decomposition.loc[shared_index].multiply(
    protein_mass_fractions.loc[shared_index,'mass fractions'], 
                                                          axis=0)

In [9]:
FDM_decomposition=FDM_decomposition.rename(columns={"EX_ac_e": "Energy|Fermentation", 
                                  "ATPM": "Energy|Respiration",
                                  "__RESIDUALS__": "No flux"})

FDM_decomposition

,Energy|Fermentation,Energy|Respiration,Sink_RNA|atp_c,Sink_RNA|ctp_c,Sink_RNA|gtp_c,Sink_RNA|utp_c,Sink_Protein|ala__L_c,Sink_Protein|arg__L_c,Sink_Protein|asn__L_c,Sink_Protein|asp__L_c,...,Sink_Others|ribflv_c,Sink_Others|sheme_c,Sink_Others|so4_c,Sink_Others|thf_c,Sink_Others|thmpp_c,Sink_Others|udcpdp_c,Sink_Others|zn2_c,Sink_Others|gthrd_c,No flux,__MIX_FUNCTIONS__
b0002,8.654303e-16,1.985343e-16,4.807531e-16,4.932551e-16,9.168445e-16,9.853240e-16,1.082729e-15,1.060910e-15,6.046455e-16,6.405738e-16,...,4.269017e-18,4.273714e-18,4.270801e-18,4.269867e-18,4.269051e-18,4.269724e-18,0.000000e+00,0.000000,2.532401e-16,5.081851e-14
b0003,1.261503e-14,1.149472e-18,0.000000e+00,0.000000e+00,0.000000e+00,9.189471e-15,2.007956e-14,3.405276e-14,5.408312e-15,0.000000e+00,...,3.669847e-14,1.398816e-13,3.506733e-14,6.654507e-08,1.345322e-14,1.093845e-14,3.273088e-14,0.000000,1.189294e-16,2.234383e-12
b0004,3.676960e-18,3.583341e-18,0.000000e+00,0.000000e+00,0.000000e+00,5.043051e-17,1.294895e-16,1.140261e-16,0.000000e+00,0.000000e+00,...,8.154400e-18,8.154732e-18,8.154526e-18,8.154460e-18,8.154402e-18,8.154450e-18,0.000000e+00,0.000000,5.579620e-18,2.619423e-15
b0007,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b0008,2.553102e-13,1.640845e-04,6.422618e-05,5.231358e-05,9.100739e-05,4.598314e-05,0.000000e+00,2.853025e-13,2.773015e-05,3.931886e-05,...,5.180115e-08,9.496922e-08,2.584988e-13,6.906838e-08,6.043567e-08,2.342291e-08,0.000000e+00,0.000009,5.970563e-16,5.630620e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4481,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.170000e-05,0.000000e+00
b4485,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b4513,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
b4515,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00


In [10]:
GO_genes=pd.concat([protein_mass_fractions.loc[non_FDM_index,'mass fractions'],
                    FDM_decomposition['No flux']], axis=0).sort_index()
GO_genes=GO_genes[GO_genes>1e-6]

GO_genes

b0006    0.000106
b0009    0.000058
b0014    0.005736
b0015    0.000338
b0019    0.000053
           ...   
b4536    0.000021
b4537    0.000065
b4554    0.000027
b4555    0.000004
b4558    0.000085
Length: 1605, dtype: float64